In [2]:
import numpy as np
import h5py as h5
import bart.python.bart as bart


In [3]:
def save(ground_truth: np.ndarray, csm: np.ndarray, mask: np.ndarray, name: str, mode = 'train'):
    with h5.File(name, 'w') as target: # creating a new file containing the necesary data to use it with MoDL
            # order: <KeysViewHDF5 ['atb', 'csm', 'mask', 'org']>
            if mode == 'train':
                #target.create_dataset('atb', data = atb_data)    # is the aliased/noisy image # It is not necessary for Brain_data
                target.create_dataset('trnCsm', data = csm)    # saves the coil sensitivity maps
                target.create_dataset('trnMask', data = mask)   # is the undersampling mask 
                target.create_dataset('trnOrg', data = ground_truth)#f['reconstruction_rss'])    # this is the original ground truth
            elif mode == 'test':
                #target.create_dataset('atb', data = atb_data)    # is the aliased/noisy image # It is not necessary for Brain_data
                target.create_dataset('tstCsm', data = csm)    # saves the coil sensitivity maps
                target.create_dataset('tstMask', data = mask)   # is the undersampling mask 
                target.create_dataset('tstOrg', data = ground_truth)#f['reconstruction_rss'])    # this is the original ground truth

In [5]:
f = h5.File('train/file_brain_AXFLAIR_201_6002906.h5', 'r')
ksp = f['kspace'] # slices, coils, h, w
#ksp = np.transpose(ksp, (0,2,3,1)) 
#ksp = np.swapaxes(ksp, 0,1) # -> coils, slices, h, w

In [ ]:
esp_maps = []
nslice = len(f['kspace'])
for sl in range(nslice):
    ksp_slice = f['kspace'][sl]
    ksp_slice = np.moveaxis(ksp_slice,0,-1)
    maps = bart.bart(1,'ecalib -m 1 -a -d 0 -g -r 24',ksp_slice[None])
    esp_maps.append(np.moveaxis(maps[0],-1,0))
esp_maps = np.stack(esp_maps,0)
print('Saving the espirit_maps of size:', esp_maps.shape)
